<a href="https://colab.research.google.com/github/mgk0422/python-basic/blob/master/final_project06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
서울_전역=pd.read_excel('/content/서울_4년_평균.xlsx')
서울_전역.head()

In [ ]:
강남_양파_dt=서울_전역[(서울_전역['지역구']=='강남구')&(서울_전역['품목']=='양파')]
강남_양파_dt.head()

In [ ]:
강남_양파_dt=강남_양파_dt[['조사일','시장']]

In [ ]:
강남_양파_dt.set_index('조사일')

In [ ]:
강남_양파_dt.info()

In [ ]:
from matplotlib import pyplot

In [ ]:
강남_양파_dt.plot(x='조사일')
plt.show()

In [ ]:
# 컬럼명 변경
강남_양파_dt.columns = ['ds', 'y']

In [ ]:
# 데이터 타입 변경
강남_양파_dt['ds']= pd.to_datetime(강남_양파_dt['ds'])

# 모델 생성
from fbprophet import Prophet
model = Prophet()

# 모델 학습
model.fit(강남_양파_dt)

In [ ]:
# train set 마지막 1년 날짜 생성
last_1year = list()
for i in range(1, 10):
    last_1year.append(['2020-%01d' % i])
last_1year = pd.DataFrame(last_1year, columns = ['ds'])
last_1year['ds']= pd.to_datetime(last_1year['ds'])

In [ ]:
# 에측
forecast = model.predict(last_1year)

In [ ]:
forecast.columns

In [ ]:
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

In [ ]:
model.plot(forecast)
pyplot.show()

In [ ]:
# train set 이후 10개월 날짜 생성
last_1year = list()
for i in range(1, 10):
    last_1year.append(['2020-%01d' % i])
last_1year = pd.DataFrame(last_1year, columns = ['ds'])
last_1year['ds']= pd.to_datetime(last_1year['ds'])

In [ ]:
forecast = model.predict(last_1year)
model.plot(forecast)
pyplot.show()

In [ ]:
# 마지막 12개월 제외시키기
train = 강남_양파_dt.drop(강남_양파_dt.index[-9:])
y_true = 강남_양파_dt['y'][-9:].values

# 모델 생성 후 학습
model = Prophet()
model.fit(train)

# train set 마지막 1년 날짜 생성
last_1year = list()
for i in range(1, 10):
    last_1year.append(['2020-%01d' % i])
last_1year = pd.DataFrame(last_1year, columns = ['ds'])
last_1year['ds']= pd.to_datetime(last_1year['ds'])

# 예측하고 비교하기
# 여기에서는 MAE를 살펴본다.
forecast = model.predict(last_1year)
y_pred = forecast['yhat'].values
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)

In [ ]:
pyplot.plot(y_true, label='Actual')
pyplot.plot(y_pred, label='Predicted')
pyplot.legend()
pyplot.show()